In this notebook, we have a dataset of user comments for youtube videos related to animals or pets. We will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.

The dataset provided for this coding test are comments for videos related to animals and/or pets. The dataset is 240MB compressed; please download the file using this google drive link:
https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

 The dataset file is comma separated, with a header line defining the field names, listed here:
● creator_name. Name of the YouTube channel creator.
● userid. Integer identifier for the users commenting on the YouTube channels.
● comment. Text of the comments made by the users.

Please use a recent version of PySpark (version 2.2 or higher) to analyze the data. Do not use
any external libraries; just use the native methods from pyspark.sql and pyspark.ml. (Do not
use pyspark.mllib as this has been deprecated.) Keep your code clean and efficient, with
enough documentation so that the grader can easily follow your train of thought. Summarize
the key results from each step. Explain how to execute your code from a command line
interface.

Step 1: Identify Cat And Dog Owners
Find the users who are cat and/or dog owners.

Step 2: Build And Evaluate Classifiers
Build classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Classify All The Users
Apply the cat/dog classifiers to all the users in the dataset. Estimate the fraction of all users
who are cat/dog owners.

Step 4: Extract Insights About Cat And Dog Owners
Find topics important to cat and dog owners.

Step 5: Identify Creators With Cat And Dog Owners In The Audience
Find creators with the most cat and/or dog owners. Find creators with the highest statistically
significant percentages of cat and/or dog owners.

link: https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

#### 0. Data Exploration and Cleaning

In [5]:
df_clean=spark.read.csv("/FileStore/tables/animals_comments.csv",inferSchema=True,header=True)
df_clean.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [6]:
df_clean.count() 

Out[ 3 ]: 5820035

In [7]:
df_clean = df_clean.na.drop(subset=["comment"])
df_clean.count()

Out[ 2 ]: 5818984

In [8]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

df_clean = df_clean.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%I have dogs%"), 1) \
                           .when(col("comment").like("%I have cats%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .otherwise(0)))

In [9]:
df_clean.show()

+--------------------+------+-------------------------------------+-----+
 creator_name|userid| comment|label|
+--------------------+------+-------------------------------------+-----+
 Doug The Pug| 87.0| I shared this to ...| 0|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0|
 bulletproof| 530.0| stop saying get e...| 0|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|
 ojatro|1031.0| I wanna see what ...| 0|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|
Hope For Paws - O...|1806.0| when I saw the en...| 0|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0|
Hope For Paws - O...|2911.0| That mother cat l...| 0|
Hope For Paws - O...|2911.0| Its people like H...| 0|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|
 Brave Wilderness|3224.0| Dont call this a ...| 0|
 MaxluvsMya|3267.0| why are you alway...| 0|
Rise Up Society F...|3372.0| Deb Tucker| 0|
 The Dodo|3466.0| Thats a deer isnt...| 0|
 Brave Wilderness|3466.0| there is no safe ...| 0|
 Brave Wilderness|3466.0| Red before yellow...| 0|
+--------------------+------+-------------------------------------+-----+
only showing top 20 rows

In [10]:
tmp = df_clean.filter(col('label')==1).count()
tmp2 = 5820035-tmp

print('number of labeled 1: '+str(tmp)+' number of labeled 0: '+str(tmp2)+' ratio : '+str(tmp/tmp2))

number of labeled 1: 40390 number of labeled 0: 5779645 ratio : 0.006988318486689062

#### 1. Data preprocessing and Build the classifier

In [12]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

word2Vec = Word2Vec(inputCol="words", outputCol="features")

In [13]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_clean)
dataset = pipelineFit.transform(df_clean)

In [14]:
dataset.count()

Out[ 7 ]: 5818984

In [15]:
dataset.show()

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 creator_name|userid| comment|label| words| features|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[0.05286521430719...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]|[-0.1149009130895...|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[0.03136705031184...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[-0.2667614513387...|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[0.09442897513508...|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[0.40065285861492...|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[0.06978420588758...|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[0.04958101763175...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| (100,[],[])|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[0.15510256215929...|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0| []| (100,[],[])|
Hope For Paws - O...|2911.0| That mother cat l...| 0|[that, mother, ca...|[-0.0189558049155...|
Hope For Paws - O...|2911.0| Its people like H...| 0|[its, people, lik...|[-0.1562179286653...|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|[steve, no, wet, ...|[0.15201501040296...|
 Brave Wilderness|3224.0| Dont call this a ...| 0|[dont, call, this...|[0.06348400722656...|
 MaxluvsMya|3267.0| why are you alway...| 0|[why, are, you, a...|[0.10792790539562...|
Rise Up Society F...|3372.0| Deb Tucker| 0| [deb, tucker]|[-0.0715384744107...|
 The Dodo|3466.0| Thats a deer isnt...| 0|[thats, a, deer, ...|[-0.0027424700558...|
 Brave Wilderness|3466.0| there is no safe ...| 0|[there, is, no, s...|[-0.0075318311505...|
 Brave Wilderness|3466.0| Red before yellow...| 0|[red, before, yel...|[-0.0010202772055...|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
only showing top 20 rows

In [16]:
(lable0_train,lable0_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(lable1_train, lable1_ex)=dataset.filter(col('label')==0).randomSplit([0.005, 0.995],seed = 100)
(lable1_test, lable1_ex2)=lable1_ex.randomSplit([0.002, 0.998],seed = 100)

In [17]:
trainingData = lable0_train.union(lable1_train)
testData=lable0_test.union(lable1_test)

In [18]:
print("Dataset Count: " + str(dataset.count()))
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Dataset Count: 5818984
Training Dataset Count: 57010
Test Dataset Count: 23398

##### LogisticRegression

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
lr = LogisticRegression(featuresCol='features', labelCol='label', predictionCol='prediction', maxIter=100, regParam=0.01)
lrModel = lr.fit(trainingData)
preds = lrModel.transform(testData)
Evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
AUC = Evaluator.evaluate(preds)
print(AUC)

0.9028048088575645

##### Parameter Tuning and K-fold cross-validation

In [23]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [24]:
lr = LogisticRegression(featuresCol='features', labelCol='label', predictionCol='prediction',regParam = 0.01)
grid =( ParamGridBuilder()
        .addGrid(lr.maxIter, [1,10,100])
        .addGrid(lr.elasticNetParam, [0,0.3])
        .build())
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator,numFolds=3)
cvModel = cv.fit(trainingData)
avg_AUC = cvModel.avgMetrics[0]
best_AUC = evaluator.evaluate(cvModel.transform(trainingData))
best_lr_model =  cvModel.bestModel



In [25]:
print('average AUC on cross validation:', avg_AUC)
print(' best model AUC on training set: ',best_AUC)

average AUC on cross validation: 0.9258583324998253
 best model AUC on training set: 0.9577573280955403

In [26]:
best_lr_model.explainParams()


Out[ 15 ]: "aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)\nelasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0, current: 0.0)\nfamily: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial. (default: auto)\nfeaturesCol: features column name (default: features, current: features)\nfitIntercept: whether to fit an intercept term (default: True)\nlabelCol: label column name (default: label, current: label)\nlowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. (undefined)\nlowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. (undefined)\nmaxIter: maximum number of iterations (>= 0) (default: 100, current: 10)\npredictionCol: prediction column name (default: prediction, current: prediction)\nprobabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities (default: probability)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)\nregParam: regularization parameter (>= 0) (default: 0.0, current: 0.01)\nstandardization: whether to standardize the training features before fitting the model (default: True)\nthreshold: threshold in binary classification prediction, in range [0, 1] (default: 0.5)\nthresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0 excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold (undefined)\ntol: the convergence tolerance for iterative algorithms (>= 0) (default: 1e-06)\nupperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. (undefined)\nupperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. (undefined)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0 (undefined)"

##### RandomForest

In [28]:
from pyspark.ml.classification import RandomForestClassifier

In [29]:
RF = RandomForestClassifier(featuresCol='features', labelCol='label', predictionCol='prediction')
grid =( ParamGridBuilder()
        .addGrid(RF.numTrees, [2,5,10])
        .addGrid(RF.maxDepth, [2,5])
        .build())
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=RF, estimatorParamMaps=grid, evaluator=evaluator,numFolds=3)
cvModel = cv.fit(trainingData)
avg_AUC = cvModel.avgMetrics[0]
best_AUC = evaluator.evaluate(cvModel.transform(trainingData))
best_RF_model =  cvModel.bestModel
print('average AUC on cross validation:', avg_AUC)
print(' best model AUC on training set: ',best_AUC)


average AUC on cross validation: 0.811676828140599
 best model AUC on training set: 0.9442774231831006

##### Gradient boosting

In [31]:
from pyspark.ml.classification import GBTClassifier

In [32]:
GBDT = GBTClassifier(featuresCol='features', labelCol='label', predictionCol='prediction')
grid =( ParamGridBuilder()
        .addGrid(GBDT.maxDepth, [2, 5])
        .addGrid(GBDT.maxIter, [5,10])
        .build())
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=GBDT, estimatorParamMaps=grid, evaluator=evaluator,numFolds=3)
cvModel = cv.fit(trainingData)
avg_AUC = cvModel.avgMetrics[0]
best_AUC = evaluator.evaluate(cvModel.transform(trainingData))
best_GBDT_model =  cvModel.bestModel
print('average AUC on cross validation:', avg_AUC)
print(' best model AUC on training set:  ',best_AUC)


average AUC on cross validation: 0.9034965790969125
 best model AUC on training set: 0.9510613607389249

In [33]:
best_model =best_lr_model

#### 2. Classify All The Users

In [35]:
pred_df = best_model.transform(dataset)
pred_df.show()
pred_df.createOrReplaceTempView('users')

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name|userid| comment|label| words| features| rawPrediction| probability|prediction|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[0.00684061900458...|[-3.0102264963292...|[0.04696600661676...| 1.0|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]|[-0.0200271792709...|[8.00899226695674...|[0.99966765091190...| 0.0|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[0.07863493140956...|[1.13611826671762...|[0.75696623541923...| 0.0|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[-0.0378745278964...|[6.98441477841877...|[0.99907465188890...| 0.0|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[0.07569982111454...|[1.10028972171025...|[0.75031438678099...| 0.0|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[0.02093426287174...|[1.35417249055193...|[0.79481094220740...| 0.0|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[-0.0541689221657...|[0.22713186644598...|[0.55654010526912...| 0.0|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[0.10066252851930...|[3.24681027538927...|[0.96255832509071...| 0.0|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| (100,[],[])|[1.31998420678556...|[0.78917907893093...| 0.0|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[0.02428726409561...|[5.21734672590189...|[0.99460754096678...| 0.0|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0| []| (100,[],[])|[1.31998420678556...|[0.78917907893093...| 0.0|
Hope For Paws - O...|2911.0| That mother cat l...| 0|[that, mother, ca...|[0.14575295662507...|[-2.0263523243647...|[0.11646374287521...| 1.0|
Hope For Paws - O...|2911.0| Its people like H...| 0|[its, people, lik...|[-0.0068530414253...|[1.66658503264535...|[0.84111998608450...| 0.0|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|[steve, no, wet, ...|[-0.0827719071371...|[-1.9434873389668...|[0.12526523570458...| 1.0|
 Brave Wilderness|3224.0| Dont call this a ...| 0|[dont, call, this...|[0.02315942091601...|[2.78019919740886...|[0.94159639980859...| 0.0|
 MaxluvsMya|3267.0| why are you alway...| 0|[why, are, you, a...|[-0.0512800433789...|[2.29528192206642...|[0.90848553528473...| 0.0|
Rise Up Society F...|3372.0| Deb Tucker| 0| [deb, tucker]|[0.03574913367629...|[6.13478236004731...|[0.99783848564756...| 0.0|
 The Dodo|3466.0| Thats a deer isnt...| 0|[thats, a, deer, ...|[0.13927142620086...|[1.86243817753110...|[0.86558088445424...| 0.0|
 Brave Wilderness|3466.0| there is no safe ...| 0|[there, is, no, s...|[0.03112862828899...|[0.47982317495314...|[0.61770611924009...| 0.0|
 Brave Wilderness|3466.0| Red before yellow...| 0|[red, before, yel...|[-0.0149449322444...|[3.68468833183650...|[0.97550982778988...| 0.0|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [36]:
#get ratio of cat or dog owners from all users
spark.sql("select sum(catagory)/count(*) as ratio from  (select userid, case when sum(prediction) =0 then 0 else 1 end   as catagory from users group by userid)").show()

+-------------------+
 ratio|
+-------------------+
0.19363772679325725|
+-------------------+

#### 3. Get insigts of Users

In [38]:
sql_query = """
select creator_name ,comment, label, prediction
from users
where label = 0 and prediction = 1
"""
df_3 = spark.sql(sql_query)
display(df_3)

creator_name,comment,label,prediction
Doug The Pug,I shared this to my friends and mom the were lol,0,1.0
Hope For Paws - Official Rescue Channel,That mother cat looks like my own Im guessing she is a russian blue due to her looks and unusual coping skills.,0,1.0
Talking Kitty Cat,steve: No wet food for a month!:cats immediately stop fighting:,0,1.0
Cole & Marmalade,cat drugs,0,1.0
Taylor Nicole Dean,I dont understand how you think she will make a good service dogs. SD are handled by a company here in Quebec (and given for free fully trained to people in need). For the first year they are fostered by families who expose them to as many things as possible and even then after a year the majority of them are deemed not fit for service work. They have to be ultra confident and never startled never afraid of anything etc... She seem like a good pet but the very opposite of what a service dog should be.,0,1.0
Rachel Fusaro,Im not allowed to have a dog because of money and my apartment doesnt allow dogs!! WHAT DO I DO!!!!???!?!?!?!?,0,1.0
Zak Georges Dog Training rEvolution,Chestnut is so cute. Your videos areuoer helpful for me. Ii dont have a dog yet but learning to train a dog ahead of time is really good for me.,0,1.0
2CAN.TV - Ripley the Toucan!,cooking the bird would be easier if the stove was turned on just saying,0,1.0
The Dodo,Oh my how crule,0,1.0
MonkeyBoo,where does Boo go potty?,0,1.0


#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [40]:
sql_query = """
select creator_name ,sum(prediction) /count(*) as owner_ratio
from users
group by creator_name
order by owner_ratio desc
"""
df_4 = spark.sql(sql_query)
display(df_4)

creator_name,owner_ratio
James Stein,1.0
FernDog Training,1.0
marifroggy,1.0
KL Daily,1.0
EdmondCats,1.0
SPCA of Texas,1.0
Dutch Hollow Acres,1.0
Working Woman Report,1.0
Kids CN,1.0
SHEBA Brand,1.0


In [41]:
sql_query = """
select creator_name ,sum(prediction) as owner_num
from users
group by creator_name
order by owner_num desc
"""
df_4 = spark.sql(sql_query)
display(df_4)

creator_name,owner_num
Brave Wilderness,78373.0
The Dodo,61045.0
Taylor Nicole Dean,41380.0
Brian Barczyk,39915.0
Robin Seplut,39724.0
Hope For Paws - Official Rescue Channel,29445.0
Gohan The Husky,23157.0
Vet Ranch,19228.0
Gone to the Snow Dogs,17671.0
Cole & Marmalade,15381.0


#### 5. Analysis and Future work

From project to your CV format 
1. overview of project 
2. data clean and modeling 
3. data analysis 
4. build ml model
5. recommendation based on the model results